In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential, layers
from keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from time import time

In [2]:
#gpu memory growth fix
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
              tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [70]:
# Loading data
train_data = pd.read_csv('E:\Python\data\Titanic\prs_train.csv')
train_data
test_data = pd.read_csv('E:\Python\data\Titanic\prs_test.csv')
test_data
# train and test split
train_X, train_y = train_data.values[:, 2:], train_data.values[:, 1]
test_X = test_data.values[:, 1:]
# One hot encoding
enc = OneHotEncoder(sparse=False, handle_unknown='ignore')
train_X_enc = enc.fit_transform(train_X)
test_X_enc = enc.transform(test_X)
# change type of data for nn to work
train_X_enc=np.asarray(train_X_enc).astype(np.float32)
train_y=np.asarray(train_y).astype(np.float32)
test_X_enc=np.asarray(test_X_enc).astype(np.float32)

In [85]:
# determine the number of input features
n_features = train_X_enc.shape[1]
# define model
model = Sequential()
model.add(layers.Dense(50, kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(layers.BatchNormalization())
model.add(layers.Activation('elu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(40, kernel_initializer='he_normal'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('elu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
#callbacks
# simple early stopping
# es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=TOL, verbose=VERBOSE, patience=N_ITER_NO_CHANGE)
# mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=VERBOSE, save_best_only=True)

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
# fit the model
model.fit(train_X_enc, train_y, epochs=100, batch_size=16, callbacks=[tensorboard,])
# make a prediction
yhat = model.predict(test_X_enc)

Epoch 1/100
56/56 [==============================] - 2s 27ms/step - loss: 0.7667 - accuracy: 0.5939
Epoch 2/100
56/56 [==============================] - 0s 6ms/step - loss: 0.6633 - accuracy: 0.6704
Epoch 3/100
56/56 [==============================] - 0s 6ms/step - loss: 0.5563 - accuracy: 0.7233
Epoch 4/100
56/56 [==============================] - 0s 8ms/step - loss: 0.5530 - accuracy: 0.7379
Epoch 5/100
56/56 [==============================] - 0s 7ms/step - loss: 0.5591 - accuracy: 0.7402
Epoch 6/100
56/56 [==============================] - 0s 7ms/step - loss: 0.5067 - accuracy: 0.7683
Epoch 7/100
56/56 [==============================] - 0s 6ms/step - loss: 0.5150 - accuracy: 0.7728
Epoch 8/100
56/56 [==============================] - 0s 6ms/step - loss: 0.5002 - accuracy: 0.7739
Epoch 9/100
56/56 [==============================] - 0s 6ms/step - loss: 0.4725 - accuracy: 0.8020
Epoch 10/100
56/56 [==============================] - 0s 7ms/step - loss: 0.4771 - accuracy: 0.8009
Epoch 11

56/56 [==============================] - 0s 7ms/step - loss: 0.2351 - accuracy: 0.9156
Epoch 80/100
56/56 [==============================] - 0s 7ms/step - loss: 0.2811 - accuracy: 0.8965
Epoch 81/100
56/56 [==============================] - 0s 6ms/step - loss: 0.2774 - accuracy: 0.8965
Epoch 82/100
56/56 [==============================] - 0s 6ms/step - loss: 0.2365 - accuracy: 0.9134
Epoch 83/100
56/56 [==============================] - 0s 6ms/step - loss: 0.2367 - accuracy: 0.9168
Epoch 84/100
56/56 [==============================] - 0s 6ms/step - loss: 0.2888 - accuracy: 0.8954
Epoch 85/100
56/56 [==============================] - 0s 6ms/step - loss: 0.2492 - accuracy: 0.9021
Epoch 86/100
56/56 [==============================] - 0s 6ms/step - loss: 0.2387 - accuracy: 0.9033
Epoch 87/100
56/56 [==============================] - 0s 7ms/step - loss: 0.2579 - accuracy: 0.9033
Epoch 88/100
56/56 [==============================] - 0s 6ms/step - loss: 0.2478 - accuracy: 0.9145
Epoch 89/100


In [86]:
z = zip(test_data.iloc[:, 0],np.argmax(yhat, axis=1))
data3 = pd.DataFrame(z, columns=['PassengerId', 'Survived'])
data3

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [87]:
# saving results
data3.to_csv('E:/Python/data/Titanic/gender_submission10.csv', index=False)